<a href="https://colab.research.google.com/github/M-Abbaszadeh/CNN/blob/main/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [31]:
import torch
import numpy as np
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import matplotlib
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dataset = MNIST(root='data/', download=True, transform=ToTensor())
len(dataset)
dataset[10]

image, label=dataset[0]

train_ds, val_ds=random_split(dataset, [50000,10000])

len(train_ds), len(val_ds)

batch_size=128

train_loader=DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True )
val_loader=DataLoader(val_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True )

for images, _ in train_loader:
    print ('image.shape:/n', image.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1,2,0))) #what it does is that it take a batch of images and convert them into a single image.
    break

for images, labels in train_loader:
    print ('image.shape:', images.shape)
    inputs=images.reshape(-1, 784)
    print('inputs.shape:', inputs.shape)
    break


In [33]:
input_size=inputs.shape[-1] #784
hidden_size=32

#layer1 generates wights and biases
layer1=nn.Linear(input_size, hidden_size) #(784, 32)
#inputs.shape

#layer1 contains wight and biases. so, we need to insert the images to layer1. the outputs essentially should be [128,32]
layer1_outputs=layer1(inputs)
print('layer1_outputs.shape:', layer1_outputs.shape)

layer1_outputs_direct= inputs @ layer1.weight.t() + layer1.bias
layer1_outputs_direct.shape

#torch.allclose is to compare the values of output directly calculated from the nn.linear and direct computation
torch.allclose(layer1_outputs, layer1_outputs_direct, 1e-3)

layer1_outputs.shape: torch.Size([128, 32])


True

In [34]:
# Activation function
#Rectified linear unit (ReLU): relu(x)=max(0,x), see example below

F.relu(torch.tensor([[1,-1,0],[-0.1,.2,-.3]]))


relu_outputs=F.relu(layer1_outputs)  #[128, 32]

relu_outputs.shape

output_size=10

layer2=nn.Linear(hidden_size, output_size) #[32 10]

layer2_outputs= layer2(relu_outputs)  #[128 32] x [32 10] + [128 10]= [128 10]  10 outputs for each image
#print (layer2_outputs.shape)

F.cross_entropy(layer2_outputs, labels)

tensor(2.3425, grad_fn=<NllLossBackward0>)

In [35]:
class MnistModel(nn.Module):
    """Feedforward neural network with 1 hidden layer"""
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        #hidden layer
        self.linear1= nn.Linear(in_size, hidden_size)
        #output layer
        self.linear2= nn.Linear(hidden_size, out_size)


    def forward(self, xb):
        # Flatten the image tensor
        xb = xb.view(xb.size(0), -1)
        # Get intermdeiate outputs using hidden layer
        out = self.linear1(xb)
        # Apply activation function
        out=F.relu(out)
        # Get prediction using output layer
        out=self.linear2(out)
        return out

    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [36]:
def accuracy(outputs, labels):
    _, preds= torch.max(outputs, dim=1)
    return torch.tensor (torch.sum (preds == labels).item()/len(preds))

In [37]:
input_size=748
hidden_size=32
num_classes=10

model=MnistModel(input_size, hidden_size=32, out_size=num_classes)


for t in model.parameters():
    print(t.shape)

torch.Size([32, 748])
torch.Size([32])
torch.Size([10, 32])
torch.Size([10])


In [39]:
torch.cuda.is_available()

True